# Modularity, Objects, and State

In [1]:
; core framework of Streams
; delay and force
(define (memo-proc proc)
  (let ((already-run? #f) (result #f))
    (lambda ()
      (if (not already-run?)
          (begin (set! result (proc))
                 (set! already-run? #t)
                 result)
          result))))
(define (delay exp)
  (memo-proc (lambda () exp)))
(define (force delayed-object)
  (delayed-object))
; pair
; cons
(define (cons-stream a b)
  (cons a (delay b)))
; car
(define (stream-car stream) (car stream))
; cdr
(define (stream-cdr stream) (force (cdr stream)))
;assert null?
(define (stream-null? s) (null? s))
; null-value
(define the-empty-stream '())
; kth-element
(define (stream-ref s n)
  (if (= n 0)
      (stream-car s)
      (stream-ref (stream-cdr s) (- n 1))))
; utilities
(define (stream-for-each proc s)
  (if (stream-null? s)
      'done
      (begin (proc (stream-car s))
             (stream-for-each proc (stream-cdr s)))))
(define (stream-enumerate-interval low high)
  (if (> low high)
      the-empty-stream
      (cons-stream
       low
       (stream-enumerate-interval (+ low 1) high))))
(define (stream-filter pred stream)
  (cond ((stream-null? stream) the-empty-stream)
        ((pred (stream-car stream))
         (cons-stream (stream-car stream)
                      (stream-filter pred
                                     (stream-cdr stream))))
        (else (stream-filter pred (stream-cdr stream)))))
; print tools
(define (display-line x)
  (display x)
  (newline))
(define (show x)
  (display-line x)
  x)
(define (display-stream s)
  (stream-for-each display-line s))

## Exercise 3.50
Complete the following definition, which generalizes stream-map to allow procedures that take multiple arguments, analogous to map in section 2.2.3, footnote 12.
```racket
(define (stream-map proc . argstreams)
  (if (<??> (car argstreams))
      the-empty-stream
      (<??>
       (apply proc (map <??> argstreams))
       (apply stream-map
              (cons proc (map <??> argstreams))))))
```

### answer

In [2]:
(define (stream-map proc . argstreams)
  (if (stream-null? (car argstreams))
      the-empty-stream
      (cons-stream
       (apply proc (map stream-car argstreams))
       (apply stream-map
              (cons proc (map stream-cdr argstreams))))))

### test

In [3]:
; a (1 2) stream list add to a (3 4) stream list
; get a (4 6) stream list
(define sl 
        (stream-map + 
                    (cons-stream 1 (cons-stream 2 the-empty-stream))
                    (cons-stream 3 (cons-stream 4 the-empty-stream))))
(display-stream sl)

4
6


'done

## Exercise 3.51
In order to take a closer look at delayed evaluation, we will use the following procedure, which simply returns its argument after printing it:
```racket
(define (show x)
  (display-line x)
  x)
```
What does the interpreter print in response to evaluating each expression in the following sequence?
```racket
(define x (stream-map show (stream-enumerate-interval 0 10)))
(stream-ref x 5)
(stream-ref x 7)
```

In [4]:
(define x (stream-map show (stream-enumerate-interval 0 10)))

0
1
2
3
4
5
6
7
8
9
10


In [5]:
(stream-ref x 5)

5

In [6]:
(stream-ref x 7)

7

Print `0` to `10`

`(stream-ref x 5)` get the 5th element of the stream

`(stream-ref x 7)` get the 7th element of the stream

## Exercise 3.52
Consider the sequence of expressions
```racket
(define sum 0)
(define (accum x)
  (set! sum (+ x sum))
  sum)
(define seq (stream-map accum (stream-enumerate-interval 1 20)))
(define y (stream-filter even? seq))
(define z (stream-filter (lambda (x) (= (remainder x 5) 0))
                         seq))
(stream-ref y 7)
(display-stream z)
```
What is the value of `sum` after each of the above expressions is evaluated? What is the printed response to evaluating the `stream-ref` and `display-stream` expressions? Would these responses differ if we had implemented `(delay <exp>)` simply as `(lambda () <exp>)` without using the optimization provided by `memo-proc` ? Explain.

In [7]:
(define sum 0)
(define (accum x)
  (set! sum (+ x sum))
  sum)
(define seq (stream-map accum (stream-enumerate-interval 1 20)))
(define y (stream-filter even? seq))
(define z (stream-filter (lambda (x) (= (remainder x 5) 0))
                         seq))

In [8]:
sum

210

In [9]:
(stream-ref y 7)

136

In [10]:
(display-stream z)

10
15
45
55
105
120
190
210


'done

In [11]:
(define (delay exp) (lambda () exp))

In [12]:
(define sum 0)
(define (accum x)
  (set! sum (+ x sum))
  sum)
(define seq (stream-map accum (stream-enumerate-interval 1 20)))
(define y (stream-filter even? seq))
(define z (stream-filter (lambda (x) (= (remainder x 5) 0))
                         seq))

In [13]:
sum

210

In [14]:
(stream-ref y 7)

136

In [15]:
(display-stream z)

10
15
45
55
105
120
190
210


'done

In [16]:
; reset delay
(define (delay exp)
  (memo-proc (lambda () exp)))

`sum` will be 210.
They will be the same, either with or without optimization.